In [1]:
import numpy as np
from numpy import linalg as la

## Unconstraied Optimization Method: Backtracking Linesearch

In [76]:
def backtracking(method,fun,mu_k,lambda_k,xk,pk,alpha=1,eta=0.01,tau=0.5):
    """
    Compute the stepsize in line search method
    
    """
    if method == "penalty method":
        lambda_k = 0
    a = alpha
    F1 = eval(fun)(xk+a*pk,mu_k,lambda_k)[0]
    [F2,G] = eval(fun)(xk,mu_k,lambda_k)

    while (F1 > F2 + eta * a * np.dot(G,pk)):
        
        a = tau * a
        F1 = eval(fun)(xk+a*pk,mu_k,lambda_k)[0]
        [F2,G] = eval(fun)(xk,mu_k,lambda_k) 
    
    return a

def uncMIN(fun,x0,mu_k,lambda_k,maxit,tol,method):

    """
    Minimizing function using backtracking-Armijo linesearch

    Input:
    Fun: string that holds the name of python function.
    x0: initial guess
    maxit:  maximum number of iterations allowe
    tol: final stopping tolerance

    Output:
    x: final iterate
    F: functional value on final iterate
    G: Gradient
    H: Hessian
    iter: total number of iterations
    status: 0 if the tolerance is obtained, 1 otherwise 
    """ 
    if method == "penalty method":
        lambda_k = 0
    iter_ = 0
    status = 1
    x = x0
    [F0,G0] = eval(fun)(x,mu_k,lambda_k)

    F = F0
    G = G0


    while la.norm(G) > tol * max(1,la.norm(G0)) and iter_ < maxit:
        iter_ += 1
        ### search direction ###
        p = -G 


        ### stepsize using backtracking-Armijo ###
        a = backtracking(method,fun,mu_k,lambda_k,x,p)
        x = x + a*p 
        [F,G] = eval(fun)(x,mu_k,lambda_k)
        print("iter:",iter_,"xk:",x,"fk:",F,"gk",la.norm(G))
        
        
    if (la.norm(G) <= tol * la.norm(G0)):
        status = 0
    
    return  [x,F,G,iter_,status]

## Penalty Method
Consider the optimization problem:
$$min\quad x+\sqrt 3y$$
$$ s.t\quad x^2+y^2=1$$ 

The optimal is $x^*=(-1/2,-\sqrt 3/2)$, the corresponding Lagrangian multiplier is $\lambda^*=1$


In [14]:
def obj_penalty(x_k,mu_k,lambda_k):
    x = x_k[0]
    y = x_k[1]
    #objective value
    F = x + np.sqrt(3)*y + 0.5 * mu_k * (x**2+y**2-1)**2
    #Gradient
    G1 = 1 + 2 * mu_k * (x**2+y**2-1) * x
    G2 = np.sqrt(3) + 2 * mu_k * (x**2+y**2-1) * y
    G = np.array([G1,G2])
    return [F,G]

def penalty_method(tol,mu_k,x_k,max_iter,lambda_k=0):
    for i in range(max_iter):
        x_k = uncMIN("obj_penalty",x_k,mu_k,lambda_k,500,tol,"penalty method")[0]
        print("iteration:",i+1,"x_k: ",x_k,"mu_k: ",mu_k)
        print("-----------")
        mu_k = mu_k*1.5
    return x_k

tol = 1e-5
mu_0 = 20
x_0 = np.array([-0.5,0])
penalty_method(tol,mu_0,x_0,10)


iteration: 1 x_k:  [-0.51200886 -0.88694274] mu_k:  20
-----------
iteration: 2 x_k:  [-0.50813368 -0.88011367] mu_k:  30.0
-----------
iteration: 3 x_k:  [-0.50546548 -0.87549215] mu_k:  45.0
-----------
iteration: 4 x_k:  [-0.50366323 -0.87237055] mu_k:  67.5
-----------
iteration: 5 x_k:  [-0.50245098 -0.87027083] mu_k:  101.25
-----------
iteration: 6 x_k:  [-0.50163795 -0.86886261] mu_k:  151.875
-----------
iteration: 7 x_k:  [-0.50109374 -0.86791997] mu_k:  227.8125
-----------
iteration: 8 x_k:  [-0.50072993 -0.86728983] mu_k:  341.71875
-----------
iteration: 9 x_k:  [-0.50048695 -0.86686898] mu_k:  512.578125
-----------
iteration: 10 x_k:  [-0.50032477 -0.86658807] mu_k:  768.8671875
-----------


array([-0.50032477, -0.86658807])

## Augmented Lagrangian
Problem: 
$$ min \quad x_1+x_2$$
$$ s.t \quad x_1^2+x_2^2-2=0$$
Solution:
$(-1,1)$


In [38]:
def Lag(x_k,mu_k,lambda_k):
    x1 = x_k[0]
    x2 = x_k[1]
    #objective value
    F = x1 + x2 - lambda_k*(x1**2+x2**2-2) + 0.5 * mu_k * (x1**2+x2**2-2)**2
    #Gradient
    G1 = 1 - 2*lambda_k*x1 + 2*mu_k*(x1**2+x2**2-2)*x1
    G2 = 1 - 2*lambda_k*x2 + 2*mu_k*(x1**2+x2**2-2)*x2
    G = np.array([G1,G2])
    return [F,G]
def aug_Lag(tol_k,mu_k,lambda_k,x_k,max_iter):
    for i in range(max_iter):
        x_k= uncMIN("Lag",x_k,mu_k,lambda_k,500,tol_k,"augmented Lagrangian")[0]
        print("lamb",lambda_k)
        print("mu_k",mu_k)
        print("iteration:",i+1,"x_k: ",x_k)
        print("-----------")
        lambda_k = lambda_k - mu_k * (x_k[0]**2 + x_k[1]**2 - 2)
        mu_k = mu_k * 1.5
        
    return x_k

tol = 1e-5
mu_0 = 10
lambda_0 = 1
x_0 = np.array([5,-5])
print("------------------")
print("solution:",aug_Lag(1e-5,mu_0,lambda_0,x_0,10))

------------------
lamb 1
mu_k 10
iteration: 1 x_k:  [-1.07451143 -0.99616319]
-----------
lamb -0.46915919940523665
mu_k 15.0
iteration: 2 x_k:  [-1.00053855 -1.00048126]
-----------
lamb -0.4997614383884361
mu_k 22.5
iteration: 3 x_k:  [-1.00000809 -0.99999723]
-----------
lamb -0.5000008544208054
mu_k 33.75
iteration: 4 x_k:  [-1.00000539 -0.99999461]
-----------
lamb -0.5000012391043355
mu_k 50.625
iteration: 5 x_k:  [-1.00000539 -0.99999461]
-----------
lamb -0.5000018161296305
mu_k 75.9375
iteration: 6 x_k:  [-1.00000532 -0.99999468]
-----------
lamb -0.5000013210435958
mu_k 113.90625
iteration: 7 x_k:  [-1.00000532 -0.99999468]
-----------
lamb -0.5000005784145438
mu_k 170.859375
iteration: 8 x_k:  [-1.00000532 -0.99999468]
-----------
lamb -0.4999994644709658
mu_k 256.2890625
iteration: 9 x_k:  [-1.00000532 -0.99999468]
-----------
lamb -0.4999977935555988
mu_k 384.43359375
iteration: 10 x_k:  [-1.00000489 -0.99999511]
-----------
solution: [-1.00000489 -0.99999511]


## Compressed Sensing
Problem:
$$ min\quad 1^Tz$$
$$ s.t\quad \phi z-y=0$$
$$ z\geq0$$

In [69]:
x=np.zeros(10)
idx=np.random.randint(10, size=4)
val=np.random.random(size=4)
x[idx]=val
phi=np.random.normal(size=(15,10))
x0=x 
y0=phi@x 

### Penalty Method

In [85]:
def obj_penalty(x_k,mu_k,lambda_k):
    g = np.empty(10)
    #objective value
    F = np.sum(x_k) + 0.5*mu_k*(la.norm(np.sum(phi@x_k-y0))**2) + 0.5*mu_k*(la.norm(np.maximum(x_k,np.array([0]*10)))**2)
    #Gradient
    for i in range(10):
        if x_k[i]>=0:
            g[i] = 0
        else:
            g[i] = 2*x_k[i]
    G = np.array([1]*10) + mu_k*(phi.T@phi@x_k-phi.T@y0) + 0.5*mu_k*g
    return [F,G]
def penalty_method(tol,mu_k,x_k,max_iter,lambda_k=0):
    for i in range(max_iter):
        x_k = uncMIN("obj_penalty",x_k,mu_k,lambda_k,500,tol,"penalty method")[0]
        print("iteration:",i+1,"x_k: ",x_k,"mu_k: ",mu_k)
        print("-----------")
        mu_k = mu_k*1.5
    return x_k
#penalty_method(1e-5,10,np.array([0]*10),20)

Results:

Discussion: The augmented Lagrangian method can avoid the problem of ill-conditioned Hessain matrix when $\mu$ is large, so it converges faster than penalty method